In [ ]:
!pip install rouge-score nltk pandas matplotlib seaborn

In [ ]:
import json
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List, Tuple
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

class QAEvaluator:
    def __init__(self):
        """Initialise l'évaluateur avec les scorers ROUGE"""
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing_function = SmoothingFunction().method1

    def simple_tokenize(self, text: str) -> List[str]:
        """Tokenisation simple sans NLTK (backup pour Colab)"""
        import re
        # Tokenisation basique
        text = text.lower()
        # Séparer les mots et supprimer la ponctuation
        tokens = re.findall(r'\b\w+\b', text)
        return tokens

    def calculate_bleu_score(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calcule les scores BLEU-1 et BLEU-4"""
        try:
            # Essayer avec NLTK d'abord
            reference_tokens = nltk.word_tokenize(reference.lower())
            candidate_tokens = nltk.word_tokenize(candidate.lower())
        except:
            # Fallback vers tokenisation simple
            reference_tokens = self.simple_tokenize(reference)
            candidate_tokens = self.simple_tokenize(candidate)

        # BLEU-1
        bleu1 = sentence_bleu([reference_tokens], candidate_tokens,
                             weights=(1, 0, 0, 0),
                             smoothing_function=self.smoothing_function)

        # BLEU-4
        bleu4 = sentence_bleu([reference_tokens], candidate_tokens,
                             weights=(0.25, 0.25, 0.25, 0.25),
                             smoothing_function=self.smoothing_function)

        return {
            'bleu1': bleu1,
            'bleu4': bleu4
        }

    def calculate_rouge_score(self, reference: str, candidate: str) -> Dict[str, float]:
        """Calcule les scores ROUGE"""
        scores = self.rouge_scorer.score(reference, candidate)

        return {
            'rouge1_f': scores['rouge1'].fmeasure,
            'rouge1_p': scores['rouge1'].precision,
            'rouge1_r': scores['rouge1'].recall,
            'rouge2_f': scores['rouge2'].fmeasure,
            'rouge2_p': scores['rouge2'].precision,
            'rouge2_r': scores['rouge2'].recall,
            'rougeL_f': scores['rougeL'].fmeasure,
            'rougeL_p': scores['rougeL'].precision,
            'rougeL_r': scores['rougeL'].recall
        }

    def evaluate_single_qa(self, reference: str, answers: Dict[str, str]) -> Dict[str, Dict[str, float]]:
        """Évalue toutes les réponses pour une question"""
        results = {}

        for model_name, answer in answers.items():
            bleu_scores = self.calculate_bleu_score(reference, answer)
            rouge_scores = self.calculate_rouge_score(reference, answer)

            results[model_name] = {**bleu_scores, **rouge_scores}

        return results

    def evaluate_dataset(self, dataset: List[Dict]) -> pd.DataFrame:
        """Évalue tout le dataset et retourne un DataFrame"""
        all_results = []

        for i, qa_pair in enumerate(dataset):
            question = qa_pair['question']
            reference = qa_pair['reference_answer']

            # Préparer les réponses à évaluer
            answers = {
                'graphrag': qa_pair['graphrag_answer'],
                'chatgpt': qa_pair['chatgpt_answer'],
                'claude': qa_pair['claude_answer'],
                'Qwen': qa_pair['Qwen_answer']
            }

            # Évaluer cette question
            qa_results = self.evaluate_single_qa(reference, answers)

            # Formater les résultats
            for model_name, scores in qa_results.items():
                result_row = {
                    'question_id': i,
                    'question': question,
                    'model': model_name,
                    **scores
                }
                all_results.append(result_row)

        return pd.DataFrame(all_results)

    def generate_summary_stats(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Génère les statistiques résumées par modèle"""
        summary = results_df.groupby('model').agg({
            'bleu1': ['mean', 'std'],
            'bleu4': ['mean', 'std'],
            'rouge1_f': ['mean', 'std'],
            'rouge2_f': ['mean', 'std'],
            'rougeL_f': ['mean', 'std']
        }).round(4)

        # Aplatir les colonnes multi-niveau
        summary.columns = [f"{col[0]}_{col[1]}" for col in summary.columns]

        return summary

    def plot_comparison(self, results_df: pd.DataFrame, save_path: str = None):
        """Crée des visualisations comparatives"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Performance Comparison: GraphRAG vs LLMs', fontsize=16)

        metrics = ['bleu1', 'bleu4', 'rouge1_f', 'rouge2_f', 'rougeL_f']
        metric_names = ['BLEU-1', 'BLEU-4', 'ROUGE-1 F1', 'ROUGE-2 F1', 'ROUGE-L F1']

        for i, (metric, name) in enumerate(zip(metrics, metric_names)):
            row = i // 3
            col = i % 3

            sns.boxplot(data=results_df, x='model', y=metric, ax=axes[row, col])
            axes[row, col].set_title(f'{name}')
            axes[row, col].set_xlabel('Modèle')
            axes[row, col].set_ylabel('Score')
            axes[row, col].tick_params(axis='x', rotation=45)

        # Supprimer le dernier subplot vide
        fig.delaxes(axes[1, 2])

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        plt.show()

    def save_results(self, results_df: pd.DataFrame, summary_df: pd.DataFrame,
                    results_path: str = "detailed_results.csv",
                    summary_path: str = "summary_results.csv"):
        """Sauvegarde les résultats"""
        results_df.to_csv(results_path, index=False)
        summary_df.to_csv(summary_path, index=True)
        print(f"Résultats détaillés sauvegardés dans: {results_path}")
        print(f"Résumé sauvegardé dans: {summary_path}")

# Utilisation
def main():
    dataset = [
        {
            "question": " What are the first warning signs and symptoms of type 2 diabetes?",
            "reference_answer": " A lot of people don't get any signs or symptoms of type 2 diabetes – or don't notice them. Symptoms can include: \n-\tWeeing a lot, especially at night. \n-\tBeing really thirsty. \n-\tFeeling more tired than usual.  \n-\tLosing weight without trying to – and getting thinner. \n-\tGenital itching or thrush. \n-\tCuts and wounds taking longer to heal.  \n-\tBlurred eyesight. ",
            "graphrag_answer": " The first warning signs and symptoms of type 2 diabetes can be subtle and may not always be immediately noticeable. However, according to the textual medical context and graph-based knowledge, some common early warning signs and symptoms include:\n\n1. **Polydipsia**: Excessive thirst, which can be a result of high blood glucose levels causing the body to produce more urine, leading to dehydration.\n2. **Polyuria**: Frequent urination, as the body tries to eliminate excess glucose through urine.\n3. **Unexplained weight loss**: Despite increased hunger and food intake, people with undiagnosed type 2 diabetes may experience weight loss due to the body's inability to effectively use insulin, leading to a breakdown of muscle and fat for energy.\n\nOther potential early warning signs and symptoms may include:\n\n* Increased fatigue or weakness\n* Blurred vision\n* Slow healing of cuts and wounds\n* Tingling or numbness in the hands and feet\n* Recurring skin, gum, or bladder infections\n\nIt's essential to note that many people with prediabetes or early type 2 diabetes may not exhibit noticeable symptoms, making regular screening and monitoring crucial for early detection and intervention.",
            "chatgpt_answer": " The first warning signs of type 2 diabetes often appear gradually and can be subtle, which is why many people don't realize they have it until blood sugar is quite high. The most common early symptoms include:\nClassic Early Signs (related to high blood sugar):\n•\tIncreased thirst (polydipsia) – feeling unusually thirsty, drinking more than usual.\n•\tFrequent urination (polyuria) – especially noticeable at night.\n•\tIncreased hunger (polyphagia) – feeling hungry soon after eating.\n•\tUnexplained weight loss – despite eating more (less common in type 2 than type 1, but can happen)\nOther Early Symptoms:\n•\tFatigue and low energy – due to cells not getting enough glucose.\n•\tBlurred vision – caused by fluid shifts in the eye lens from high blood sugar.\n•\tSlow-healing wounds or frequent infections – high glucose impairs healing and increases infection risk.\n•\tTingling, burning, or numbness in hands/feet – early nerve damage (diabetic neuropathy).\n•\tDarkened patches of skin (acanthosis nigricans) – usually in neck, armpits, or groin; often linked to insulin resistance.\n•\tRecurrent yeast or urinary tract infections – due to sugar-rich environment for microbes.",
            "claude_answer": " The early warning signs and symptoms of type 2 diabetes can be subtle and develop gradually, which is why many people don't realize they have the condition initially. Here are the key symptoms to watch for:\nCommon Early Symptoms:\nIncreased thirst and frequent urination - Your kidneys work overtime to filter excess glucose, leading to more urination and subsequent dehydration that triggers thirst.\nUnexplained fatigue - When cells can't effectively use glucose for energy due to insulin resistance, you may feel persistently tired or weak.\nBlurred vision - High blood sugar can cause fluid to be pulled from tissues, including the lenses of your eyes, affecting your ability to focus.\nSlow-healing cuts or frequent infections - Elevated glucose levels can impair your immune system and blood circulation, making it harder for wounds to heal.\nIncreased hunger - Despite eating, your cells aren't getting adequate glucose, which can trigger persistent hunger.\nUnexplained weight loss - Though less common in type 2 diabetes than type 1, some people may lose weight as their body starts burning muscle and fat for energy.\nTingling or numbness in hands or feet - High blood sugar can damage nerves, particularly in extremities.\nDarkened skin patches - Called acanthosis nigricans, these velvety dark patches often appear in body folds like the neck, armpits, or groin.\nImportant Notes:\nMany people with type 2 diabetes have no symptoms initially, or symptoms are so mild they're overlooked. This is why regular screening is crucial, especially if you have risk factors like family history, obesity, or are over 45.",
            "Qwen_answer": " The first warning signs and symptoms of type 2 diabetes can be subtle and may develop gradually, which is why many people have it for years without knowing. Common early symptoms include:\n1.\tIncreased Thirst (Polydipsia): Feeling unusually thirsty and drinking more fluids than usual.\n2.\tFrequent Urination (Polyuria): Needing to urinate more often, especially at night, as the kidneys work to remove excess sugar from the blood.\n3.\tIncreased Hunger (Polyphagia): Feeling hungry even after eating, because cells aren't getting enough glucose for energy.\n4.\tUnexplained Weight Loss: Losing weight despite eating more, as the body starts to break down muscle and fat for energy when it can't use glucose properly.\n5.\tFatigue: Feeling unusually tired or sluggish due to cells not receiving adequate glucose for energy.\n6.\tBlurred Vision: High blood sugar can cause fluid to be pulled from the lenses of the eyes, affecting the ability to focus.\n7.\tSlow-Healing Sores or Frequent Infections: High blood sugar can impair circulation and weaken the immune system, leading to slower healing and more infections (e.g., urinary tract, skin, or yeast infections).\n8.\tTingling, Numbness, or Pain in Hands or Feet (Neuropathy): Early nerve damage from prolonged high blood sugar levels.\n9.\tDarkened Skin Patches (Acanthosis Nigricans): Velvety, darkened skin, often in the neck, armpits, or groin, which can be a sign of insulin resistance.\n10.\tFrequent Yeast Infections: Especially in women, due to elevated sugar levels promoting yeast growth.\nBecause symptoms develop slowly, they may be overlooked or attributed to other causes. People at higher risk—such as those who are overweight, physically inactive, have a family history of diabetes, or are over age 45—should consider regular screening even if symptoms are mild or absent."
        },
        {
            "question": " Can type 2 diabetes be prevented?",
            "reference_answer": "Type 2 diabetes can sometimes be prevented because unlike type 1 diabetes which can't yet be prevented - your body is still making the hormone insulin. And it is the insulin which helps manage blood sugar levels.  Research has shown for some people, getting support to make changes to your lifestyle including healthy eating, moving more and losing weight, can help reduce your risk of developing type 2 diabetes by about half.  In some cases, if you're at risk of type 2 diabetes losing any excess weight, keeping active and eating healthily, can help your body make more insulin or use the insulin more effectively",
            "graphrag_answer": " Type 2 diabetes can be prevented or delayed through lifestyle interventions, including a healthy diet, physical activity, and weight control, particularly in high-risk individuals. Randomized controlled trials (RCTs) have consistently shown that these interventions are effective in preventing or delaying the onset of type 2 diabetes and managing cardiovascular risk factors.\n\nAccording to the textual medical context, national diabetes prevention programs in countries such as Finland and China have been successful in helping people sustain healthy lifestyle changes, which has significantly impacted health-related quality of life. These programs have also contributed to reducing the risk of developing type 2 diabetes.\n\nThe graph-based knowledge summary supports this finding, with a high graph density and strong relation filtering, indicating a robust connection between lifestyle interventions and the prevention of type 2 diabetes.\n\nIt is essential to note that the development and progression of type 2 diabetes involve a complex interplay of genetic, epigenetic, lifestyle, demographic, socioeconomic, therapeutic, and environmental factors. Therefore, while lifestyle interventions are crucial, they should be tailored to individual needs and circumstances.\n\nThe American Diabetes Association and other clinical guidelines recommend the following preventive measures:\n\n1. Maintaining a healthy weight through a balanced diet and regular physical activity.\n2. Engaging in at least 150 minutes of moderate-intensity aerobic exercise, or 75 minutes of vigorous-intensity aerobic exercise, or a combination of both, per week.\n3. Eating a healthy, balanced diet that is low in saturated fats, added sugars, and sodium.\n4. Getting regular check-ups and screenings for blood glucose and other cardiovascular risk factors.\n\nWhile there are uncertainties regarding the optimal preventive strategies for specific individuals, the available evidence suggests that lifestyle interventions are a critical component of type 2 diabetes prevention. By combining healthy lifestyle habits with regular monitoring and screenings, individuals can significantly reduce their risk of developing type 2 diabetes and its related complications.\n\nIn summary, type 2 diabetes can be prevented or delayed through lifestyle interventions, including a healthy diet, physical activity, and weight control. National diabetes prevention programs and clinical guidelines support these findings, and individuals can reduce their risk by adopting healthy lifestyle habits and getting regular check-ups and screenings.",
            "chatgpt_answer": " Yes — in many cases, type 2 diabetes (T2D) can be delayed or even prevented, especially in people at high risk (for example, those who are overweight, have a family history, or have prediabetes).\nHere's how prevention works:\n🔹 Key Strategies to Prevent Type 2 Diabetes\n1.\tWeight management\no\tEven a modest weight loss (5–7% of body weight) can significantly reduce risk.\no\tExample: A person weighing 80 kg losing just 4–6 kg can see measurable benefits.\n2.\tHealthy diet\no\tPrioritize whole grains, vegetables, fruits, legumes, nuts, and lean protein.\no\tLimit sugary drinks, processed foods, and refined carbs.\no\tMediterranean-style and DASH diets are proven beneficial.\n3.\tRegular physical activity\no\tAt least 150 minutes/week of moderate activity (e.g., brisk walking, cycling).\no\tExercise improves insulin sensitivity and helps regulate blood sugar.\n4.\tManaging other risk factors\no\tControl blood pressure and cholesterol.\no\tAvoid smoking, as it increases insulin resistance and cardiovascular risk.\n5.\tMedical support (if needed)\no\tPeople with prediabetes may be prescribed metformin if lifestyle changes alone aren't enough, especially if they are young, obese, or have gestational diabetes history.",
            "claude_answer": "Type 2 diabetes can often be prevented or significantly delayed through lifestyle modifications, especially for people at higher risk. Here's what the research shows:\nPrevention is possible for many people because type 2 diabetes typically develops gradually over years, giving opportunities for intervention. The strongest evidence comes from major studies like the Diabetes Prevention Program, which showed that lifestyle changes reduced diabetes risk by 58% in high-risk individuals.\nKey prevention strategies include:\nWeight management is particularly important - even modest weight loss of 5-10% can substantially reduce risk. Regular physical activity helps improve insulin sensitivity and glucose metabolism. The recommendation is at least 150 minutes of moderate exercise weekly, but even smaller amounts provide benefits.\nDietary changes focus on reducing refined carbohydrates and added sugars while emphasizing whole grains, vegetables, lean proteins, and healthy fats. The Mediterranean diet pattern has shown particular promise for diabetes prevention.\nRisk factors you can't change include genetics, age, ethnicity, and having had gestational diabetes. However, people with these risk factors can still significantly reduce their likelihood of developing type 2 diabetes through the modifiable factors.\nWho benefits most from prevention efforts are those with prediabetes (elevated but not yet diabetic blood sugar levels), family history of diabetes, obesity, or metabolic syndrome. Regular screening helps identify people in this higher-risk category.\nThe earlier someone starts these preventive measures, the more effective they tend to be, but it's never too late to reduce risk through healthy lifestyle changes.",
            "Qwen_answer": "Yes, type 2 diabetes can often be prevented or delayed through lifestyle changes, especially for people at high risk. While certain risk factors like age, family history, and ethnicity cannot be changed, many others can be managed effectively. Key preventive strategies include:\n1.\tMaintaining a Healthy Weight: Excess body weight, particularly around the abdomen, increases insulin resistance. Losing even a small amount of weight (5–10% of body weight) can significantly reduce the risk.\n2.\tRegular Physical Activity: Aim for at least 150 minutes of moderate-intensity exercise (like brisk walking) per week. Exercise helps the body use insulin more efficiently and lowers blood glucose levels.\n3.\tHealthy Eating: Focus on a balanced diet rich in:\n•\tWhole grains\n•\tFruits and vegetables\n•\tLean proteins\n•\tHealthy fats Limit intake of processed foods, sugary drinks, and saturated fats.\n4.\tAvoiding Tobacco and Limiting Alcohol: Smoking increases the risk of type 2 diabetes and complications. Excessive alcohol consumption can also contribute to weight gain and blood sugar imbalances.\n5.\tRegular Health Screenings: Early detection of prediabetes (higher-than-normal blood sugar levels) allows for timely intervention. Blood tests like fasting glucose or HbA1c can identify risk before diabetes develops.\n6.\tManaging Stress and Sleep: Chronic stress and poor sleep patterns can affect blood sugar control and increase diabetes risk. Practicing good sleep hygiene and stress-reduction techniques (like mindfulness or yoga) can help.\nStudies such as the Diabetes Prevention Program (DPP) have shown that lifestyle interventions can reduce the risk of developing type 2 diabetes by up to 58% in high-risk individuals."
        },
        {
            "question": " What are three ways to prevent type 2 diabetes?",
            "reference_answer": " The three main ways to prevent type 2 diabetes are:  \nEating well\nMoving more\nLosing weight if you are living with obesity or overweight or have an unhealthy waist size",
            "Qwen_answer": " Preventing Type 2 Diabetes (T2D) is crucial, given its significant impact on public health worldwide. Here are three evidence-based ways to prevent Type 2 Diabetes:\n\n1. **Healthy Eating and Weight Management**: Maintaining a healthy diet and weight is essential in preventing T2D. This can be achieved through energy-restricted diets and sustainable lifestyle changes. Weight loss has been associated with improved glycemic control and cardiometabolic health among individuals with prediabetes and T2D. National diabetes prevention programs in countries like Finland and China have shown that focusing on lifestyle modification, including diet, can contribute to reducing diabetes risk.\n\n2. **Regular Physical Activity**: Engaging in regular physical activity is another critical aspect of T2D prevention. Increased levels of physical activity, combined with energy-restricted diets, are recommended for preventing obesity and T2D. Programs that promote behaviors such as exercise have been effective in helping individuals sustain healthy lifestyle changes, thereby attenuating the incidence of diabetes.\n\n3. **Managing Risk Factors and Monitoring Health**: Controlling blood sugar levels, managing risk factors such as blood pressure and lipid levels, and monitoring for the development of complications are also vital. While the textual context does not explicitly mention vitamin D supplementation as a prevention method, research into its effects on insulin resistance and potential role in T2D prevention is ongoing. However, current clinical guidelines primarily emphasize lifestyle modifications, including diet and physical activity, as the cornerstone of T2D prevention.",
            "chatgpt_answer": " Here are three effective ways to help prevent type 2 diabetes:\n1.\tMaintain a Healthy Weight\no\tExcess body fat, especially around the abdomen, increases insulin resistance.\no\tEven modest weight loss (5–7% of body weight) can significantly reduce the risk.\n2.\tAdopt a Balanced Diet\no\tPrioritize whole grains, vegetables, fruits, lean proteins, and healthy fats.\no\tLimit sugary drinks, refined carbs, and processed foods that cause blood sugar spikes.\n3.\tStay Physically Active\no\tAim for at least 150 minutes of moderate exercise per week (e.g., brisk walking, cycling, swimming).\no\tRegular activity improves insulin sensitivity and helps control weight.\n👉 Additional helpful habits include not smoking, limiting alcohol, and getting regular check-ups if you have risk factors (family history, obesity, sedentary lifestyle).\nWould you like me to expand this into a patient-friendly guide with daily lifestyle tips, or keep it as a short clinical summary?",
            "claude_answer": " Maintain a healthy weight and lose excess weight if needed. Even modest weight loss of 5-10% of body weight can significantly reduce diabetes risk. Excess weight, particularly around the abdomen, increases insulin resistance and makes it harder for your body to regulate blood sugar effectively.\nEngage in regular physical activity. Aim for at least 150 minutes of moderate-intensity exercise per week, such as brisk walking, swimming, or cycling. Physical activity helps your muscles use glucose more efficiently, improves insulin sensitivity, and supports weight management. Both aerobic exercise and resistance training are beneficial.\nFollow a balanced, nutrient-rich diet. Focus on whole foods like vegetables, fruits, whole grains, lean proteins, and healthy fats while limiting processed foods, sugary drinks, and refined carbohydrates. This eating pattern helps maintain stable blood sugar levels and supports a healthy weight. The Mediterranean diet and similar approaches have shown particularly strong evidence for diabetes prevention.\nThese lifestyle modifications work synergistically - combining all three approaches is more effective than focusing on just one. Studies like the Diabetes Prevention Program have shown that these changes can reduce type 2 diabetes risk by up to 58%.",
            "graphrag_answer": " Three effective ways to prevent type 2 diabetes are:\n1.\tMaintain a Healthy Diet: Focus on eating whole grains, lean proteins, healthy fats, fruits, and vegetables while limiting processed foods, sugary drinks, and refined carbohydrates. A balanced diet helps regulate blood sugar and insulin levels.\n2.\tEngage in Regular Physical Activity: Aim for at least 150 minutes of moderate-intensity exercise (like brisk walking) per week. Physical activity helps the body use insulin more efficiently and supports weight management.\n3.\tAchieve and Maintain a Healthy Weight: Losing even a small amount of weight—5% to 7% of body weight—can significantly reduce the risk of developing type 2 diabetes, especially for people who are overweight or obese.\nThese lifestyle changes are supported by research, including the Diabetes Prevention Program (DPP), which showed they can cut the risk of type 2 diabetes by more than half."
        },
        {
            "question": " What is the main treatment for type 2 diabetes?",
            "reference_answer": " The main treatments for managing blood sugar levels if you have type 2 diabetes are: \n-\tEating well and moving more \n-\tWeight loss  \n-\tMetformin (usually taken as a tablet)  \n-\tInsulin along with other medication that helps lower blood sugar levels \n-\tOther types of diabetes medicine that is injected or taken as tablets.  \n-\tWeight loss surgery \n-\tSupport with your emotional health \nUsing a flash glucose monitor or continuous glucose monitor can also make it easier for people with type 2 diabetes to monitor their blood sugar levels without having to do so many finger prick checks. This tech isn't offered to everyone with type 2 diabetes.",
            "graphrag_answer": " The main treatment for type 2 diabetes - focuses on preventing and managing cardiovascular disease, heart failure, and chronic kidney disease, while maintaining glycemic control. \n- Lifestyle modifications, such as Eating well and moving more, Weight loss  \n- Pharmacotherapy, including medications such Metformin  as sodium–glucose-transporter-2 (SGLT2) inhibitors or glucagon-like peptide-1 (GLP-1) receptor agonists, and ultimately insulin therapy. \nGlycemic control is generally defined as a hemoglobin A1c (HbA1c) level of 7% or less. \nNote: Information on specific treatment plans may vary depending on individual patient needs and clinical guidelines.",
            "chatgpt_answer": " The main treatment for type 2 diabetes usually begins with lifestyle changes, and if needed, progresses to medications and sometimes insulin.\nHere's the standard approach:\n1.\tLifestyle modifications (first-line treatment)\no\tHealthy diet: balanced meals, reduced refined carbs and sugars, high in fiber, lean protein, and healthy fats.\no\tRegular physical activity: at least 150 minutes of moderate exercise per week (e.g., brisk walking, cycling).\no\tWeight management: losing even 5–10% of body weight can improve blood sugar control.\no\tSmoking cessation and limiting alcohol.\n2.\tMedications (when lifestyle alone is not enough)\no\tMetformin is the first-line drug for most people. It helps the body use insulin better and lowers glucose production in the liver.\no\tOther classes may be added if needed, such as SGLT2 inhibitors, GLP-1 receptor agonists, DPP-4 inhibitors, or sulfonylureas, depending on the patient's condition, risk of complications, and other health factors.\n3.\tInsulin therapy\no\tSometimes needed if blood sugar remains high despite oral medicines, or in cases of advanced diabetes or during acute illness.\n👉 In summary: Lifestyle modification is the foundation, metformin is the main medication, and insulin or other drugs may be added if needed.",
            "claude_answer": " The main treatment for type 2 diabetes is lifestyle modification, specifically:\nDiet and nutrition management - This involves controlling carbohydrate intake, portion sizes, and meal timing to help regulate blood sugar levels. Many people benefit from working with a dietitian to develop a sustainable eating plan.\nRegular physical activity - Exercise helps improve insulin sensitivity and glucose uptake by muscles. Both aerobic exercise and resistance training are beneficial.\nWeight management - For those who are overweight, even modest weight loss (5-10% of body weight) can significantly improve blood sugar control.\nMedication is often needed alongside lifestyle changes. Metformin is typically the first-line medication prescribed, as it helps reduce glucose production by the liver and improves insulin sensitivity.\nFor many people, type 2 diabetes requires a combination approach - lifestyle changes form the foundation, with medications added as needed to achieve target blood sugar levels. The specific treatment plan varies based on individual factors like blood sugar levels, other health conditions, and how well someone responds to initial interventions.\nRegular monitoring of blood glucose and periodic check-ups with healthcare providers are also essential components of diabetes management to adjust treatment as needed and prevent complications.",
            "Qwen_answer": " The main treatment for type 2 diabetes typically involves a combination of lifestyle and medications, tailored to the individual's needs. The cornerstone of treatment includes:\n1.\tLifestyle Changes:\n•\tHealthy Eating: A balanced diet rich in whole grains, fruits, vegetables, lean proteins, and healthy fats, while limiting refined sugars and processed foods.\n•\tRegular Physical Activity: At least 150 minutes of moderate-intensity aerobic exercise per week (such as brisk walking), along with strength training.\n•\tWeight Management: Losing even a modest amount of weight (5–10% of body weight) can significantly improve blood glucose control.\n2.\tMedications:\n•\tMetformin is usually the first-line medication. It helps lower blood glucose by decreasing liver glucose production and improving insulin sensitivity.\n•\tOther oral or injectable medications may be added if needed, such as:\n•\tSulfonylureas\n•\tDPP-4 inhibitors\n•\tSGLT2 inhibitors\n•\tGLP-1 receptor agonists\n•\tInsulin therapy may be required in some cases, especially as the disease progresses and beta-cell function declines.\n3.\tMonitoring Blood Glucose: Regular self-monitoring helps track how well treatment is working and guides adjustments.\n4.\tManagement of Related Conditions: Controlling blood pressure, cholesterol levels, and screening for complications (e.g., eye, kidney, nerve damage) are also critical parts of care.\nThe overall goal is to maintain blood glucose levels within a target range to prevent complications and improve quality of life. Treatment plans are individualized based on the patient's health status, preferences, and risk factors."
        },
        {
            "question": " What could causes Type 2 diabetes ?",
            "reference_answer": " Type 2 diabetes is high blood sugar levels caused by a lack of a hormone called insulin. Either your body isn't making enough or the insulin it makes doesn't work properly. This is sometimes called insulin resistance.  \nLack of insulin causes glucose from what you eat or drink to build up in your blood. So you may get symptoms of type 2 diabetes.  \nIf you have enough insulin or it is working properly, the glucose in your blood should be released into your cells to give you energy.  \nIf you do not have enough insulin - or the insulin your body produces is not working properly — the glucose in your blood cannot enter your cells and give you energy. And your blood sugar levels keep rising.  \nThe main causes of type 2 diabetes are: \n-\tLiving with obesity or overweight.  \n-\tYour waist measurement is unhealthy for your gender or ethnicity  \n-\tToo much fat stored in or around your liver and pancreas – which can affect people of a healthy weight as well as people living with obesity or overweight.   \n-\tOther factors that put you at greater risk of developing type 2 diabetes include high blood pressure, age, ethnicity and family history. We call them the risk factors of type 2 diabetes. \nSome of these are things you can't change, like your age (your risk goes up as you get older), your ethnicity (people from African Caribbean, Black African or South Asian backgrounds are at higher risk – and from a younger age), and if family members have type 2 diabetes. But there are some risk factors that you may be able to change, like your weight or waist size. \nIf you live with obesity or overweight, it's harder for your body to manage the levels of sugar in your blood, and having a high waist measurement means you're more at risk of insulin resistance. Our research has linked type 2 diabetes with fat surrounding the liver and pancreas. \nNot being active for most of the day is also a risk factor. We know that moving more will help reduce your risk of developing type 2 diabetes.",
            "graphrag_answer": " Type 2 diabetes (T2D) is a complex metabolic disease that is influenced by multiple factors. The primary causes of T2D can be attributed to a combination of genetic, nutritional, and environmental factors, as well as physical activity levels.\nthe causes of Type 2 diabetes can be attributed to:\n\n1. **Genetics**: Heightened genetic susceptibility, particularly in certain ethnic groups.\n2. **Nutrition**: Poor diet, high caloric intake, and associated obesity.\n3. **Physical activity**: Decreased levels of physical activity.\n4. **Insulin resistance**: Dysregulation of insulin signaling in glucose metabolism.\n5. **Abnormal insulin secretion**: Progressive loss of β-cell function.\n\nIt is essential to note that the exact mechanisms and interactions between these factors are not yet fully understood and require further research. However, it is clear that a combination of genetic, lifestyle, and environmental factors contributes to the development of Type 2 diabetes.\n\nAs a medical assistant, I would recommend that patients focus on maintaining a healthy lifestyle, including a balanced diet, regular physical activity, and weight management, to reduce their risk of developing T2D. Additionally, patients with a family history of T2D or other risk factors should be screened regularly and managed according to clinical guidelines, which may include medications, lifestyle interventions, or a combination of both.",
            "chatgpt_answer": " Type 2 diabetes develops when the body cannot use insulin effectively (insulin resistance) and, over time, the pancreas cannot make enough insulin to keep blood sugar under control.\nThe causes are multifactorial — usually a mix of genetics, lifestyle, and metabolic changes.\nHere are the main causes and risk factors:\n________________________________________\n🔹 Biological & Genetic Causes\n•\tInsulin resistance: Muscle, liver, and fat cells do not respond properly to insulin, so glucose stays in the blood.\n•\tPancreatic beta-cell dysfunction: Over time, the pancreas cannot produce enough insulin to overcome resistance.\n•\tGenetics: Family history of type 2 diabetes increases risk, especially if a parent or sibling has it.\n•\tEthnicity: Higher prevalence in South Asian, African, Hispanic, and Indigenous populations.\n________________________________________\n🔹 Lifestyle & Environmental Causes\n•\tExcess weight/obesity: Especially visceral fat (around the abdomen), which increases insulin resistance.\n•\tPhysical inactivity: Muscles that are not active use less glucose, worsening insulin resistance.\n•\tUnhealthy diet: High intake of refined carbs, sugary drinks, and processed foods.\n•\tSleep problems: Poor sleep and conditions like sleep apnea are linked to insulin resistance.\n•\tChronic stress: Stress hormones (cortisol, adrenaline) can raise blood sugar and worsen insulin resistance.\n________________________________________\n🔹 Medical & Metabolic Causes\n•\tMetabolic syndrome: A cluster of conditions (high blood pressure, high cholesterol, central obesity).\n•\tHormonal disorders: Such as polycystic ovary syndrome (PCOS).\n•\tAging: Risk rises after age 45, though it is increasingly common in younger people.\n________________________________________\n✅ In short: Type 2 diabetes is caused by a combination of genetics + lifestyle + metabolic dysfunction. The key mechanisms are insulin resistance and progressive loss of insulin production.",
            "claude_answer": " Several factors can contribute to the development of type 2 diabetes:\nInsulin resistance is the core mechanism - when cells become less responsive to insulin, the pancreas initially produces more insulin to compensate, but over time may not keep up with demand.\nMajor risk factors include:\nObesity and excess weight - Particularly abdominal fat, which increases insulin resistance. This is one of the strongest risk factors.\nPhysical inactivity - Sedentary lifestyle reduces the body's ability to use glucose effectively and contributes to weight gain.\nGenetics and family history - Having parents or siblings with type 2 diabetes significantly increases risk, suggesting important genetic components.\nAge - Risk increases after age 35, though type 2 diabetes is increasingly seen in younger people.\nEthnicity - Higher rates occur in African Americans, Hispanic/Latino Americans, Native Americans, Pacific Islanders, and some Asian populations.\nOther contributing factors:\n•\tPrediabetes - Elevated blood sugar that hasn't reached diabetic levels yet\n•\tGestational diabetes history - Women who had diabetes during pregnancy have increased risk\n•\tPolycystic ovary syndrome (PCOS)\n•\tHigh blood pressure and abnormal cholesterol levels\n•\tPoor diet - High in processed foods, sugary drinks, and refined carbohydrates\n•\tChronic stress - Can affect hormone levels and glucose metabolism\n•\tSleep disorders - Poor sleep quality or sleep apnea may increase risk\nType 2 diabetes typically develops gradually through the interaction of genetic predisposition with environmental and lifestyle factors. While genetics can't be changed, many of the lifestyle-related risk factors are modifiable through diet, exercise, and weight management.",
            "Qwen_answer": " Type 2 diabetes is a chronic condition that occurs when the body becomes resistant to insulin or when the pancreas is unable to produce enough insulin to maintain normal blood glucose levels. Several factors contribute to the development of Type 2 diabetes, including:\n1.\tInsulin Resistance: The most common underlying cause. Cells in muscles, fat, and the liver don't respond well to insulin and can't easily take up glucose from the bloodstream.\n2.\tGenetics and Family History: Having a parent or sibling with Type 2 diabetes increases your risk. Certain genetic factors can influence insulin production and sensitivity.\n3.\tObesity and Excess Body Weight: Especially excess fat around the abdomen, is strongly linked to insulin resistance. Fat tissue, particularly visceral fat, releases substances that interfere with insulin signaling.\n4.\tPhysical Inactivity: Lack of regular physical activity contributes to weight gain and reduces the body's sensitivity to insulin.\n5.\tUnhealthy Diet: Diets high in processed foods, sugary beverages, red meats, and unhealthy fats can increase the risk. Low intake of fiber, fruits, and vegetables is also a factor.\n6.\tAge: The risk increases with age, especially after 45, due to decreased physical activity, muscle mass loss, and weight gain. However, it's becoming more common in younger individuals, including children and adolescents.\n7.\tEthnicity: Certain ethnic groups—including African Americans, Hispanics/Latinos, Native Americans, Asian Americans, and Pacific Islanders—have a higher risk.\n8.\tPrediabetes: A condition where blood sugar levels are higher than normal but not yet diabetic. Without intervention, prediabetes often progresses to Type 2 diabetes.\n9.\tGestational Diabetes: Women who had gestational diabetes during pregnancy are at higher risk of developing Type 2 diabetes later in life.\n10.\tPolycystic Ovary Syndrome (PCOS): This hormonal disorder in women is associated with insulin resistance and increases the risk of Type 2 diabetes.\n11.\tSleep Problems: Conditions like sleep apnea and chronic sleep deprivation are linked to insulin resistance and higher diabetes risk.\n12.\tMetabolic Syndrome: A cluster of conditions including high blood pressure, high blood sugar, excess abdominal fat, and abnormal cholesterol levels increases the risk.\n13.\tCertain Medications: Long-term use of steroids, some antipsychotics, and other drugs can affect blood sugar control.\n14.\tOther Health Conditions: Non-alcoholic fatty liver disease (NAFLD) and chronic inflammation are also associated with increased risk.\nWhile some risk factors like age and genetics cannot be changed, many causes—such as obesity, poor diet, and inactivity—are modifiable through lifestyle changes, which can significantly reduce the risk or delay the onset of Type 2 diabetes."
        }

    ]

    # Initialiser l'évaluateur
    evaluator = QAEvaluator()

    # Évaluer le dataset
    print("Évaluation en cours...")
    results_df = evaluator.evaluate_dataset(dataset)

    # Générer les statistiques résumées
    summary_df = evaluator.generate_summary_stats(results_df)

    # Afficher les résultats
    print("\n=== RÉSUMÉ DES PERFORMANCES ===")
    print(summary_df)

    print("\n=== TOP 3 PAR MÉTRIQUE ===")
    for metric in ['bleu1', 'bleu4', 'rouge1_f', 'rouge2_f', 'rougeL_f']:
        print(f"\n{metric.upper()}:")
        top_models = results_df.groupby('model')[metric].mean().sort_values(ascending=False).head(3)
        for model, score in top_models.items():
            print(f"  {model}: {score:.4f}")

    # Créer les visualisations
    evaluator.plot_comparison(results_df, "comparison_plots.png")

    # Sauvegarder les résultats
    evaluator.save_results(results_df, summary_df)

if __name__ == "__main__":
    main()